# Unlearning Experiments ([steps](https://www.overleaf.com/project/623aa8d6b133433e9006bc8a))

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import random

In [ ]:
# Get Fashion-MNIST data
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [ ]:
# Set batch size
batch_size = 100

# Create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [ ]:
# Get CPU or GPU device for training
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define 3-layer fully connected neural network
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
# Trains for one epoch.
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
# Prints the accuracy and average loss for the test set.
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
# Define loss function and hyperparameters.
loss_fn = nn.CrossEntropyLoss()
lr = 1e-2
batch_size = 100
num_epochs = 100

In [ ]:
# Sample U (one element) from specified label and get dataset without U.
def get_modified_training_dataloader(label, batch_size):
    # Create copy of dataset to modify
    modified_dataset = datasets.FashionMNIST(
        root="data",
        train=True,
        download=True,
        transform=ToTensor(),
    )

    # Find index of first element from class i (the single element in set of 
    # delete requests U).
    indicies_from_class = modified_dataset.targets == label
    index_of_u = int((indicies_from_class == True).nonzero()[0][0])

    # Remove the element
    indicies_without_u = [True] * len(modified_dataset)
    indicies_without_u[index_of_u] = False
    modified_dataset.targets = modified_dataset.targets[indicies_without_u]
    modified_dataset.data = modified_dataset.data[indicies_without_u]
    
    return DataLoader(modified_dataset, batch_size=batch_size)

# Verify that get_modified_training_dataloader removes just one element.
# Length should be 59999.
print(len(get_modified_training_dataloader(0, batch_size).dataset))

# Trains a neural network for a given number of epochs.
def train_model(model, train_dataloader, optimizer, test_dataloader = test_dataloader, loss_fn = loss_fn, num_epochs = 100):
    for t in range(num_epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train(train_dataloader, model, loss_fn, optimizer)
        test(test_dataloader, model, loss_fn)
    print("Done!")

59999


In [ ]:
# Initialize list of models
# models = []

# Compute w and noisy ws for each random seed. For loop in the form i, i+1 to 
# avoid problems with runtime disconnecting (each iteration takes approx 2 hrs).
for seed in range(1, 2):
    # Set seed for training on regular dataset
    random.seed(seed)
    torch.manual_seed(seed)

    # Train for w(S)
    model_regular = NeuralNetwork().to(device)
    optimizer = torch.optim.SGD(model_regular.parameters(), lr=lr)
    train_model(model_regular, train_dataloader, optimizer, num_epochs = num_epochs)

    models.append(model_regular)

    # Train for each of the w(S\U) 
    models_modified = []
    for label in range(10):
        print("Removing an element from class: " + str(label))

        # Remove an element from the training dataset
        modified_train_dataloader = get_modified_training_dataloader(label, batch_size)

        # Set the same seed to start each training iteration
        random.seed(seed)
        torch.manual_seed(seed)

        # Create and train model for the modified dataset
        model_modified = NeuralNetwork().to(device)
        optimizer = torch.optim.SGD(model_modified.parameters(), lr=lr)
        train_model(model_modified, modified_train_dataloader, optimizer, num_epochs = num_epochs)

        models_modified.append(model_modified)

    models.append(models_modified)

Streaming output truncated to the last 5000 lines.
loss: 0.248668  [10000/59999]
loss: 0.250904  [20000/59999]
loss: 0.338268  [30000/59999]
loss: 0.457174  [40000/59999]
loss: 0.398513  [50000/59999]
Test Error: 
 Accuracy: 87.2%, Avg loss: 0.362060 

Epoch 48
-------------------------------
loss: 0.201242  [    0/59999]
loss: 0.246775  [10000/59999]
loss: 0.249957  [20000/59999]
loss: 0.334991  [30000/59999]
loss: 0.456761  [40000/59999]
loss: 0.396031  [50000/59999]
Test Error: 
 Accuracy: 87.2%, Avg loss: 0.360601 

Epoch 49
-------------------------------
loss: 0.199721  [    0/59999]
loss: 0.244792  [10000/59999]
loss: 0.249333  [20000/59999]
loss: 0.332227  [30000/59999]
loss: 0.454980  [40000/59999]
loss: 0.393419  [50000/59999]
Test Error: 
 Accuracy: 87.2%, Avg loss: 0.359113 

Epoch 50
-------------------------------
loss: 0.197911  [    0/59999]
loss: 0.243194  [10000/59999]
loss: 0.248165  [20000/59999]
loss: 0.329133  [30000/59999]
loss: 0.454729  [40000/59999]
loss: 0.39

In [ ]:
# Save models (code has been changed at each iteration)
from google.colab import files

torch.save(models[2].state_dict(), "model_reg_seed_1.pt")
files.download("model_reg_seed_1.pt")
for i in range(10):
  filename = "model_removed_" + str(i) + "_seed_1.pt"
  torch.save(models[3][i].state_dict(), filename)
  files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>